### 套件載入

In [1]:
import gzip
import math
import pickle
import zlib
import io
import joblib
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

In [2]:
os.chdir("/Users/liouscott/Documents/scott/kaggle/8th/Santander Product Recommendation/santander-product-recommendation-8th-place-master/")

In [4]:
!ls -al

total 30160
drwxr-xr-x@ 15 liouscott  staff       480 Jan  7 19:49 .
drwxr-xr-x   4 liouscott  staff       128 Jan  7 15:09 ..
-rw-r--r--@  1 liouscott  staff      6148 Jan  7 17:23 .DS_Store
-rwxr-xr-x@  1 liouscott  staff      1064 Dec 22  2016 LICENSE
-rwxr-xr-x@  1 liouscott  staff       497 Dec 22  2016 README.md
drwxr-xr-x   4 liouscott  staff       128 Jan  7 16:31 __pycache__
-rwxr-xr-x@  1 liouscott  staff       870 Dec 22  2016 clean.py
-rwxr-xr-x@  1 liouscott  staff      4584 Dec 22  2016 engines.py
drwxr-xr-x   6 liouscott  staff       192 Jan  7 13:49 input
-rw-r--r--@  1 liouscott  staff    232695 Jan  8 09:59 log.txt
-rwxr-xr-x@  1 liouscott  staff     14658 Jan  7 17:22 main.py
-rw-r--r--   1 liouscott  staff  15119842 Jan  7 19:49 next_multi.pickle
-rwxr-xr-x@  1 liouscott  staff        43 Dec 22  2016 run.sh
drwxr-xr-x  12 liouscott  staff       384 Jan  8 10:01 tmp
-rwxr-xr-x@  1 liouscott  staff      2269 Dec 22  2016 utils.py


### 載入程式執行附加元件

In [3]:
import math
import time

import numpy as np

class Timer:
    def __init__(self, text=None):
        self.text = text

    def __enter__(self):
        self.cpu = time.clock()
        self.time = time.time()
        if self.text:
            print("{}...".format(self.text))
        return self

    def __exit__(self, *args):
        self.cpu = time.clock() - self.cpu
        self.time = time.time() - self.time
        if self.text:
            print("%s: cpu %0.2f, time %0.2f\n" % (self.text, self.cpu, self.time))

def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] # "2016-05-28"
    int_date = (int(Y) - 2015) * 12 + int(M)
    assert 1 <= int_date <= 12 + 6
    return int_date

# "2016-05-28" or "" or nan
def date_to_float(str_date):
    if str_date.__class__ is float and math.isnan(str_date) or str_date == "":
        return np.nan
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    float_date = float(Y) * 12 + float(M)
    return float_date


products = (
    "ind_ahor_fin_ult1",
    "ind_aval_fin_ult1",
    "ind_cco_fin_ult1" ,
    "ind_cder_fin_ult1",
    "ind_cno_fin_ult1" ,
    "ind_ctju_fin_ult1",
    "ind_ctma_fin_ult1",
    "ind_ctop_fin_ult1",
    "ind_ctpp_fin_ult1",
    "ind_deco_fin_ult1",
    "ind_deme_fin_ult1",
    "ind_dela_fin_ult1",
    "ind_ecue_fin_ult1",
    "ind_fond_fin_ult1",
    "ind_hip_fin_ult1" ,
    "ind_plan_fin_ult1",
    "ind_pres_fin_ult1",
    "ind_reca_fin_ult1",
    "ind_tjcr_fin_ult1",
    "ind_valo_fin_ult1",
    "ind_viv_fin_ult1" ,
    "ind_nomina_ult1"  ,
    "ind_nom_pens_ult1",
    "ind_recibo_ult1"  ,
)

dtypes = {
    "fecha_dato": str,
    "ncodpers": int,
    "conyuemp": str, # Spouse index. 1 if the customer is spouse of an employee
}


def apk(actual, predicted, k=10, default=1.0):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return default

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10, default=1.0):
    return np.mean([apk(a,p,k,default) for a,p in zip(actual, predicted)])

### 將處理後的資料表讀入檔案

In [4]:
train_df = pd.read_csv('input/all_clean.csv', dtype=dtypes)

/Users/liouscott/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 遺失值補零

In [5]:
for prod in products:
    train_df[prod] = train_df[prod].fillna(0.0).astype(np.int8)

### 預處理資料

In [6]:
def apply_transforms(train_df):
    features = []
    with Timer("apply transforms"):
        label_encode(train_df, features, "canal_entrada")
        #label_encode(train_df, features, "nomprov") # use cod_prov only
        label_encode(train_df, features, "pais_residencia")

        train_df["age"] = train_df["age"].fillna(0.0).astype(np.int16)
        features.append("age")

        train_df["renta"].fillna(1.0, inplace=True)
        train_df["renta_top"] = encode_top(train_df["renta"])
        assert_uniq(train_df["renta_top"], "renta_top")
        features.append("renta_top")
        train_df["renta"] = train_df["renta"].map(math.log)
        features.append("renta")

        train_df["antiguedad"] = train_df["antiguedad"].map(lambda x: 0.0 if x < 0 or math.isnan(x) else x+1.0).astype(np.int16)
        features.append("antiguedad")

        train_df["tipodom"] = train_df["tipodom"].fillna(0.0).astype(np.int8)
        features.append("tipodom")

        train_df["cod_prov"] = train_df["cod_prov"].fillna(0.0).astype(np.int8)
        features.append("cod_prov")

        train_df["fecha_dato_month"] = train_df["fecha_dato"].map(lambda x: int(x.split("-")[1])).astype(np.int8)
        features.append("fecha_dato_month")
        train_df["fecha_dato_year"] = train_df["fecha_dato"].map(lambda x: float(x.split("-")[0])).astype(np.int16)
        features.append("fecha_dato_year")
        train_df["fecha_alta_month"] = train_df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
        features.append("fecha_alta_month")
        train_df["fecha_alta_year"] = train_df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int16)
        features.append("fecha_alta_year")

        train_df["fecha_dato_float"] = train_df["fecha_dato"].map(date_to_float)
        train_df["fecha_alta_float"] = train_df["fecha_alta"].map(date_to_float)

        train_df["dato_minus_alta"] = train_df["fecha_dato_float"] - train_df["fecha_alta_float"]
        features.append("dato_minus_alta")

        train_df["int_date"] = train_df["fecha_dato"].map(date_to_int).astype(np.int8)

        custom_one_hot(train_df, features, "indresi", {"n":"N"})
        custom_one_hot(train_df, features, "indext", {"s":"S"})
        custom_one_hot(train_df, features, "conyuemp", {"n":"N"})
        custom_one_hot(train_df, features, "sexo", {"h":"H", "v":"V"})
        custom_one_hot(train_df, features, "ind_empleado", {"a":"A", "b":"B", "f":"F", "n":"N"})
        custom_one_hot(train_df, features, "ind_nuevo", {"new":1})
        custom_one_hot(train_df, features, "segmento", {"top":"01 - TOP", "particulares":"02 - PARTICULARES", "universitario":"03 - UNIVERSITARIO"})
        custom_one_hot(train_df, features, "indfall", {"s":"S"})

        train_df["ind_actividad_cliente"] = train_df["ind_actividad_cliente"].map(lambda x: 0.0 if math.isnan(x) else x+1.0).astype(np.int8)
        features.append("ind_actividad_cliente")
        custom_one_hot(train_df, features, "indrel", {"1":1, "99":99})
        train_df["indrel_1mes"] = train_df["indrel_1mes"].map(lambda x: 5.0 if x == "P" else x).astype(float).fillna(0.0).astype(np.int8)
        assert_uniq(train_df["indrel_1mes"], "indrel_1mes")
        features.append("indrel_1mes")
        custom_one_hot(train_df, features, "tiprel_1mes", {"a":"A", "i":"I", "p":"P", "r":"R"}, check=True)

    return train_df, tuple(features)

In [7]:
transformers = {}

###  轉換One Hot Encoding

In [8]:
def label_encode(df, features, name):
    df[name] = df[name].astype('str')
    if name in transformers: # test
        df[name] = transformers[name].transform(df[name])
    else: # train
        transformers[name] = LabelEncoder()
        df[name] = transformers[name].fit_transform(df[name])
    features.append(name)

### 選出出現次數出過100次的數字

In [9]:
def encode_top(s, count=100, dtype=np.int8):
    uniqs, freqs = np.unique(s, return_counts=True)
    top = sorted(zip(uniqs,freqs), key=lambda vk: vk[1], reverse = True)[:count]
    top_map = {uf[0]: l+1 for uf, l in zip(top, range(len(top)))}
    return s.map(lambda x: top_map.get(x, 0)).astype(dtype)

### 客製化One Hot Encoding，指定要轉換的名稱

In [10]:
def custom_one_hot(df, features, name, names, dtype=np.int8, check=False):
    for n, val in names.items():
        new_name = "%s_%s" % (name, n)
        print(name, new_name)
        df[new_name] = df[name].map(lambda x: 1 if x == val else 0).astype(dtype)

        if check:
            assert_uniq(df[new_name], new_name)
        features.append(new_name)

### 檢核是否有重覆值

In [11]:
def assert_uniq(series, name):
    uniq = np.unique(series, return_counts=True)
    print("assert_uniq", name, uniq)

### 製作預處理資料

In [ ]:
train_df, features = apply_transforms(train_df)

apply transforms...
assert_uniq renta_top (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int8), array([11487211,  3022340,     5936,     1854,     1584,     1495,
           1444,     1416,     1256,     1218,     1114,     1076,
           1017,      952,      941,      899,      855,      844,
            823,      813,      756,      723,      718,      712,
            694,      690,      666,      663,      661,      657,
            645,      624,      619,      618, 

### 由於要在變數中製造客戶前幾個月購買的變數，所以在用以下形式處理

In [16]:
prev_dfs = []
prod_features = None
use_features = frozenset([1,2])

In [17]:
def make_prev_df(train_df, step):
    with Timer("make prev%s DF" % step):
        prev_df = pd.DataFrame()
        prev_df["ncodpers"] = train_df["ncodpers"]
        prev_df["int_date"] = train_df["int_date"].map(lambda x: x+step).astype(np.int8)
        prod_features = ["%s_prev%s" % (prod, step) for prod in products]
        for prod, prev in zip(products, prod_features):
            prev_df[prev] = train_df[prod]
    return prev_df, tuple(prod_features)

### 製作前五個月客戶購賣變數

In [25]:
for step in range(1,6):
    prev1_train_df, prod1_features = make_prev_df(train_df, step)
    prev_dfs.append(prev1_train_df)
    if step in use_features:
        features += prod1_features
    if step == 1:
        prod_features = prod1_features

make prev1 DF...
make prev1 DF: cpu 5.77, time 5.77

make prev2 DF...
make prev2 DF: cpu 5.64, time 5.63

make prev3 DF...
make prev3 DF: cpu 5.77, time 5.77

make prev4 DF...
make prev4 DF: cpu 5.60, time 5.59

make prev5 DF...
make prev5 DF: cpu 5.60, time 5.60



### 以下為檢視過程中的值

In [20]:
train_df["int_date"].map(lambda x: x+1).astype(np.int8)

0            2
1            2
2            2
3            2
4            2
5            2
6            2
7            2
8            2
9            2
10           2
11           2
12           2
13           2
14           2
15           2
16           2
17           2
18           2
19           2
20           2
21           2
22           2
23           2
24           2
25           2
26           2
27           2
28           2
29           2
            ..
14576894    19
14576895    19
14576896    19
14576897    19
14576898    19
14576899    19
14576900    19
14576901    19
14576902    19
14576903    19
14576904    19
14576905    19
14576906    19
14576907    19
14576908    19
14576909    19
14576910    19
14576911    19
14576912    19
14576913    19
14576914    19
14576915    19
14576916    19
14576917    19
14576918    19
14576919    19
14576920    19
14576921    19
14576922    19
14576923    19
Name: int_date, Length: 14576924, dtype: int8

In [31]:
products

('ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1')

In [30]:
prod1_features

('ind_ahor_fin_ult1_prev5',
 'ind_aval_fin_ult1_prev5',
 'ind_cco_fin_ult1_prev5',
 'ind_cder_fin_ult1_prev5',
 'ind_cno_fin_ult1_prev5',
 'ind_ctju_fin_ult1_prev5',
 'ind_ctma_fin_ult1_prev5',
 'ind_ctop_fin_ult1_prev5',
 'ind_ctpp_fin_ult1_prev5',
 'ind_deco_fin_ult1_prev5',
 'ind_deme_fin_ult1_prev5',
 'ind_dela_fin_ult1_prev5',
 'ind_ecue_fin_ult1_prev5',
 'ind_fond_fin_ult1_prev5',
 'ind_hip_fin_ult1_prev5',
 'ind_plan_fin_ult1_prev5',
 'ind_pres_fin_ult1_prev5',
 'ind_reca_fin_ult1_prev5',
 'ind_tjcr_fin_ult1_prev5',
 'ind_valo_fin_ult1_prev5',
 'ind_viv_fin_ult1_prev5',
 'ind_nomina_ult1_prev5',
 'ind_nom_pens_ult1_prev5',
 'ind_recibo_ult1_prev5')

In [28]:
features

('canal_entrada',
 'pais_residencia',
 'age',
 'renta_top',
 'renta',
 'antiguedad',
 'tipodom',
 'cod_prov',
 'fecha_dato_month',
 'fecha_dato_year',
 'fecha_alta_month',
 'fecha_alta_year',
 'dato_minus_alta',
 'indresi_n',
 'indext_s',
 'conyuemp_n',
 'sexo_h',
 'sexo_v',
 'ind_empleado_a',
 'ind_empleado_b',
 'ind_empleado_f',
 'ind_empleado_n',
 'ind_nuevo_new',
 'segmento_top',
 'segmento_particulares',
 'segmento_universitario',
 'indfall_s',
 'ind_actividad_cliente',
 'indrel_1',
 'indrel_99',
 'indrel_1mes',
 'tiprel_1mes_a',
 'tiprel_1mes_i',
 'tiprel_1mes_p',
 'tiprel_1mes_r',
 'ind_ahor_fin_ult1_prev1',
 'ind_aval_fin_ult1_prev1',
 'ind_cco_fin_ult1_prev1',
 'ind_cder_fin_ult1_prev1',
 'ind_cno_fin_ult1_prev1',
 'ind_ctju_fin_ult1_prev1',
 'ind_ctma_fin_ult1_prev1',
 'ind_ctop_fin_ult1_prev1',
 'ind_ctpp_fin_ult1_prev1',
 'ind_deco_fin_ult1_prev1',
 'ind_deme_fin_ult1_prev1',
 'ind_dela_fin_ult1_prev1',
 'ind_ecue_fin_ult1_prev1',
 'ind_fond_fin_ult1_prev1',
 'ind_hip_fin_u

In [36]:
prev_dfs[4]

,ncodpers,int_date,ind_ahor_fin_ult1_prev5,ind_aval_fin_ult1_prev5,ind_cco_fin_ult1_prev5,ind_cder_fin_ult1_prev5,ind_cno_fin_ult1_prev5,ind_ctju_fin_ult1_prev5,ind_ctma_fin_ult1_prev5,ind_ctop_fin_ult1_prev5,...,ind_hip_fin_ult1_prev5,ind_plan_fin_ult1_prev5,ind_pres_fin_ult1_prev5,ind_reca_fin_ult1_prev5,ind_tjcr_fin_ult1_prev5,ind_valo_fin_ult1_prev5,ind_viv_fin_ult1_prev5,ind_nomina_ult1_prev5,ind_nom_pens_ult1_prev5,ind_recibo_ult1_prev5
0,1375586,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1050611,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1050612,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1050613,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1050614,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1050615,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1050616,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1050617,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1050619,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1050620,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 將資料合併回原本的training data

In [43]:
def join_with_prev(df, prev_df, how):
    with Timer("join %s" % how):
        assert set(df.columns.values.tolist()) & set(prev_df.columns.values.tolist()) == set(["ncodpers", "int_date"])
        print("before join", len(df))
        df = df.merge(prev_df, on=["ncodpers", "int_date"], how=how)
        for f in set(prev_df.columns.values.tolist()) - set(["ncodpers", "int_date"]):
            df[f] = df[f].astype(np.float16)
        print("after join", len(df))
        return df

In [44]:
for i, prev_df in enumerate(prev_dfs):
    with Timer("join train with prev%s" % (i+1)):
        how = "inner" if i == 0 else "left"
        train_df = join_with_prev(train_df, prev_df, how=how)

join train with prev1...
join inner...
before join 14576924
after join 13612036
join inner: cpu 31.39, time 31.44

join train with prev1: cpu 31.95, time 32.01

join train with prev2...
join left...
before join 13612036
after join 13612036
join left: cpu 34.98, time 36.07

join train with prev2: cpu 35.66, time 36.74

join train with prev3...
join left...
before join 13612036
after join 13612036
join left: cpu 38.01, time 38.50

join train with prev3: cpu 38.75, time 39.25

join train with prev4...
join left...
before join 13612036
after join 13612036
join left: cpu 41.62, time 42.30

join train with prev4: cpu 42.44, time 43.11

join train with prev5...
join left...
before join 13612036
after join 13612036
join left: cpu 46.97, time 47.76

join train with prev5: cpu 47.92, time 48.71



### 以下為過程中的值

In [37]:
set(prev_dfs[0].columns.values.tolist()) - set(["ncodpers", "int_date"])

{'ind_ahor_fin_ult1_prev1',
 'ind_aval_fin_ult1_prev1',
 'ind_cco_fin_ult1_prev1',
 'ind_cder_fin_ult1_prev1',
 'ind_cno_fin_ult1_prev1',
 'ind_ctju_fin_ult1_prev1',
 'ind_ctma_fin_ult1_prev1',
 'ind_ctop_fin_ult1_prev1',
 'ind_ctpp_fin_ult1_prev1',
 'ind_deco_fin_ult1_prev1',
 'ind_dela_fin_ult1_prev1',
 'ind_deme_fin_ult1_prev1',
 'ind_ecue_fin_ult1_prev1',
 'ind_fond_fin_ult1_prev1',
 'ind_hip_fin_ult1_prev1',
 'ind_nom_pens_ult1_prev1',
 'ind_nomina_ult1_prev1',
 'ind_plan_fin_ult1_prev1',
 'ind_pres_fin_ult1_prev1',
 'ind_reca_fin_ult1_prev1',
 'ind_recibo_ult1_prev1',
 'ind_tjcr_fin_ult1_prev1',
 'ind_valo_fin_ult1_prev1',
 'ind_viv_fin_ult1_prev1'}

In [39]:
set(train_df.columns.values.tolist()) & set(prev_dfs[0].columns.values.tolist())

{'int_date', 'ncodpers'}

In [40]:
set(train_df.columns.values.tolist())

{'age',
 'antiguedad',
 'canal_entrada',
 'cod_prov',
 'conyuemp',
 'conyuemp_n',
 'dato_minus_alta',
 'fecha_alta',
 'fecha_alta_float',
 'fecha_alta_month',
 'fecha_alta_year',
 'fecha_dato',
 'fecha_dato_float',
 'fecha_dato_month',
 'fecha_dato_year',
 'ind_actividad_cliente',
 'ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_empleado',
 'ind_empleado_a',
 'ind_empleado_b',
 'ind_empleado_f',
 'ind_empleado_n',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_nom_pens_ult1',
 'ind_nomina_ult1',
 'ind_nuevo',
 'ind_nuevo_new',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_recibo_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'indext',
 'indext_s',
 'indfall',
 'indfall_s',
 'indrel',
 'indrel_1',

In [41]:
set(prev_dfs[0].columns.values.tolist())

{'ind_ahor_fin_ult1_prev1',
 'ind_aval_fin_ult1_prev1',
 'ind_cco_fin_ult1_prev1',
 'ind_cder_fin_ult1_prev1',
 'ind_cno_fin_ult1_prev1',
 'ind_ctju_fin_ult1_prev1',
 'ind_ctma_fin_ult1_prev1',
 'ind_ctop_fin_ult1_prev1',
 'ind_ctpp_fin_ult1_prev1',
 'ind_deco_fin_ult1_prev1',
 'ind_dela_fin_ult1_prev1',
 'ind_deme_fin_ult1_prev1',
 'ind_ecue_fin_ult1_prev1',
 'ind_fond_fin_ult1_prev1',
 'ind_hip_fin_ult1_prev1',
 'ind_nom_pens_ult1_prev1',
 'ind_nomina_ult1_prev1',
 'ind_plan_fin_ult1_prev1',
 'ind_pres_fin_ult1_prev1',
 'ind_reca_fin_ult1_prev1',
 'ind_recibo_ult1_prev1',
 'ind_tjcr_fin_ult1_prev1',
 'ind_valo_fin_ult1_prev1',
 'ind_viv_fin_ult1_prev1',
 'int_date',
 'ncodpers'}

In [47]:
train_df[(train_df.ncodpers == 1050617) ]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1_prev5,ind_plan_fin_ult1_prev5,ind_pres_fin_ult1_prev5,ind_reca_fin_ult1_prev5,ind_tjcr_fin_ult1_prev5,ind_valo_fin_ult1_prev5,ind_viv_fin_ult1_prev5,ind_nomina_ult1_prev5,ind_nom_pens_ult1_prev5,ind_recibo_ult1_prev5
613468,2015-02-28,1050617,N,36,H,23,2012-08-10,0.0,36,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213815,2015-03-28,1050617,N,36,H,23,2012-08-10,0.0,36,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1869507,2015-04-28,1050617,N,36,H,23,2012-08-10,0.0,36,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1881487,2015-05-28,1050617,N,36,H,23,2012-08-10,0.0,36,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2535384,2015-06-28,1050617,N,36,H,23,2012-08-10,0.0,36,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3595222,2015-07-28,1050617,N,36,H,23,2012-08-10,0.0,36,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4277255,2015-08-28,1050617,N,36,H,23,2012-08-10,0.0,37,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5193899,2015-09-28,1050617,N,36,H,23,2012-08-10,0.0,38,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5608918,2015-10-28,1050617,N,36,H,23,2012-08-10,0.0,39,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7157786,2015-11-28,1050617,N,36,H,23,2012-08-10,0.0,40,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 利用商品的變數，擴增新的標準差的變數

In [60]:
for prod in products:
    print()
    print(prod)
    for begin, end in [(1,3),(1,5),(2,5)]:
        prods = ["%s_prev%s" % (prod, i) for i in range(begin,end+1)]
        print (prods)
        
        mp_df = train_df.as_matrix(columns=prods)
        #print(prods)

        stdf = "%s_std_%s_%s" % (prod,begin,end)
        print (stdf)
        
        train_df[stdf] = np.nanstd(mp_df, axis=1) #  * prev1_bin

        features += (stdf,)


ind_ahor_fin_ult1
['ind_ahor_fin_ult1_prev1', 'ind_ahor_fin_ult1_prev2', 'ind_ahor_fin_ult1_prev3']
ind_ahor_fin_ult1_std_1_3
['ind_ahor_fin_ult1_prev1', 'ind_ahor_fin_ult1_prev2', 'ind_ahor_fin_ult1_prev3', 'ind_ahor_fin_ult1_prev4', 'ind_ahor_fin_ult1_prev5']
ind_ahor_fin_ult1_std_1_5
['ind_ahor_fin_ult1_prev2', 'ind_ahor_fin_ult1_prev3', 'ind_ahor_fin_ult1_prev4', 'ind_ahor_fin_ult1_prev5']
ind_ahor_fin_ult1_std_2_5


/Users/liouscott/anaconda/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)



ind_aval_fin_ult1
['ind_aval_fin_ult1_prev1', 'ind_aval_fin_ult1_prev2', 'ind_aval_fin_ult1_prev3']
ind_aval_fin_ult1_std_1_3
['ind_aval_fin_ult1_prev1', 'ind_aval_fin_ult1_prev2', 'ind_aval_fin_ult1_prev3', 'ind_aval_fin_ult1_prev4', 'ind_aval_fin_ult1_prev5']
ind_aval_fin_ult1_std_1_5
['ind_aval_fin_ult1_prev2', 'ind_aval_fin_ult1_prev3', 'ind_aval_fin_ult1_prev4', 'ind_aval_fin_ult1_prev5']
ind_aval_fin_ult1_std_2_5

ind_cco_fin_ult1
['ind_cco_fin_ult1_prev1', 'ind_cco_fin_ult1_prev2', 'ind_cco_fin_ult1_prev3']
ind_cco_fin_ult1_std_1_3
['ind_cco_fin_ult1_prev1', 'ind_cco_fin_ult1_prev2', 'ind_cco_fin_ult1_prev3', 'ind_cco_fin_ult1_prev4', 'ind_cco_fin_ult1_prev5']
ind_cco_fin_ult1_std_1_5
['ind_cco_fin_ult1_prev2', 'ind_cco_fin_ult1_prev3', 'ind_cco_fin_ult1_prev4', 'ind_cco_fin_ult1_prev5']
ind_cco_fin_ult1_std_2_5

ind_cder_fin_ult1
['ind_cder_fin_ult1_prev1', 'ind_cder_fin_ult1_prev2', 'ind_cder_fin_ult1_prev3']
ind_cder_fin_ult1_std_1_3
['ind_cder_fin_ult1_prev1', 'ind_cder_fin

ind_viv_fin_ult1_std_1_5
['ind_viv_fin_ult1_prev2', 'ind_viv_fin_ult1_prev3', 'ind_viv_fin_ult1_prev4', 'ind_viv_fin_ult1_prev5']
ind_viv_fin_ult1_std_2_5

ind_nomina_ult1
['ind_nomina_ult1_prev1', 'ind_nomina_ult1_prev2', 'ind_nomina_ult1_prev3']
ind_nomina_ult1_std_1_3
['ind_nomina_ult1_prev1', 'ind_nomina_ult1_prev2', 'ind_nomina_ult1_prev3', 'ind_nomina_ult1_prev4', 'ind_nomina_ult1_prev5']
ind_nomina_ult1_std_1_5
['ind_nomina_ult1_prev2', 'ind_nomina_ult1_prev3', 'ind_nomina_ult1_prev4', 'ind_nomina_ult1_prev5']
ind_nomina_ult1_std_2_5

ind_nom_pens_ult1
['ind_nom_pens_ult1_prev1', 'ind_nom_pens_ult1_prev2', 'ind_nom_pens_ult1_prev3']
ind_nom_pens_ult1_std_1_3
['ind_nom_pens_ult1_prev1', 'ind_nom_pens_ult1_prev2', 'ind_nom_pens_ult1_prev3', 'ind_nom_pens_ult1_prev4', 'ind_nom_pens_ult1_prev5']
ind_nom_pens_ult1_std_1_5
['ind_nom_pens_ult1_prev2', 'ind_nom_pens_ult1_prev3', 'ind_nom_pens_ult1_prev4', 'ind_nom_pens_ult1_prev5']
ind_nom_pens_ult1_std_2_5

ind_recibo_ult1
['ind_recibo

### 以下為觀察變化的值

In [57]:
mp_df = train_df.as_matrix(columns=['ind_ahor_fin_ult1_prev1', 'ind_ahor_fin_ult1_prev2', 'ind_ahor_fin_ult1_prev3'])

In [58]:
np.nanstd(mp_df, axis=1)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float16)

### 加變數前

In [59]:
features

('canal_entrada',
 'pais_residencia',
 'age',
 'renta_top',
 'renta',
 'antiguedad',
 'tipodom',
 'cod_prov',
 'fecha_dato_month',
 'fecha_dato_year',
 'fecha_alta_month',
 'fecha_alta_year',
 'dato_minus_alta',
 'indresi_n',
 'indext_s',
 'conyuemp_n',
 'sexo_h',
 'sexo_v',
 'ind_empleado_a',
 'ind_empleado_b',
 'ind_empleado_f',
 'ind_empleado_n',
 'ind_nuevo_new',
 'segmento_top',
 'segmento_particulares',
 'segmento_universitario',
 'indfall_s',
 'ind_actividad_cliente',
 'indrel_1',
 'indrel_99',
 'indrel_1mes',
 'tiprel_1mes_a',
 'tiprel_1mes_i',
 'tiprel_1mes_p',
 'tiprel_1mes_r',
 'ind_ahor_fin_ult1_prev1',
 'ind_aval_fin_ult1_prev1',
 'ind_cco_fin_ult1_prev1',
 'ind_cder_fin_ult1_prev1',
 'ind_cno_fin_ult1_prev1',
 'ind_ctju_fin_ult1_prev1',
 'ind_ctma_fin_ult1_prev1',
 'ind_ctop_fin_ult1_prev1',
 'ind_ctpp_fin_ult1_prev1',
 'ind_deco_fin_ult1_prev1',
 'ind_deme_fin_ult1_prev1',
 'ind_dela_fin_ult1_prev1',
 'ind_ecue_fin_ult1_prev1',
 'ind_fond_fin_ult1_prev1',
 'ind_hip_fin_u

###  加變數後

In [61]:
features

('canal_entrada',
 'pais_residencia',
 'age',
 'renta_top',
 'renta',
 'antiguedad',
 'tipodom',
 'cod_prov',
 'fecha_dato_month',
 'fecha_dato_year',
 'fecha_alta_month',
 'fecha_alta_year',
 'dato_minus_alta',
 'indresi_n',
 'indext_s',
 'conyuemp_n',
 'sexo_h',
 'sexo_v',
 'ind_empleado_a',
 'ind_empleado_b',
 'ind_empleado_f',
 'ind_empleado_n',
 'ind_nuevo_new',
 'segmento_top',
 'segmento_particulares',
 'segmento_universitario',
 'indfall_s',
 'ind_actividad_cliente',
 'indrel_1',
 'indrel_99',
 'indrel_1mes',
 'tiprel_1mes_a',
 'tiprel_1mes_i',
 'tiprel_1mes_p',
 'tiprel_1mes_r',
 'ind_ahor_fin_ult1_prev1',
 'ind_aval_fin_ult1_prev1',
 'ind_cco_fin_ult1_prev1',
 'ind_cder_fin_ult1_prev1',
 'ind_cno_fin_ult1_prev1',
 'ind_ctju_fin_ult1_prev1',
 'ind_ctma_fin_ult1_prev1',
 'ind_ctop_fin_ult1_prev1',
 'ind_ctpp_fin_ult1_prev1',
 'ind_deco_fin_ult1_prev1',
 'ind_deme_fin_ult1_prev1',
 'ind_dela_fin_ult1_prev1',
 'ind_ecue_fin_ult1_prev1',
 'ind_fond_fin_ult1_prev1',
 'ind_hip_fin_u

In [62]:
train_df

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_viv_fin_ult1_std_2_5,ind_nomina_ult1_std_1_3,ind_nomina_ult1_std_1_5,ind_nomina_ult1_std_2_5,ind_nom_pens_ult1_std_1_3,ind_nom_pens_ult1_std_1_5,ind_nom_pens_ult1_std_2_5,ind_recibo_ult1_std_1_3,ind_recibo_ult1_std_1_5,ind_recibo_ult1_std_2_5
0,2015-02-28,545090,N,36,V,67,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
1,2015-02-28,545106,N,36,H,40,2005-08-30,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
2,2015-02-28,545103,N,36,V,45,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
3,2015-02-28,545102,N,36,V,46,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
4,2015-02-28,545101,N,36,V,38,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
5,2015-02-28,545158,N,36,V,47,2005-08-03,0.0,109,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
6,2015-02-28,545100,N,36,V,43,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
7,2015-02-28,545099,N,36,V,45,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
8,2015-02-28,545092,N,36,V,66,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN
9,2015-02-28,545091,N,36,V,41,2005-08-02,0.0,121,1.0,...,NaN,0.000000,0.00000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN


### ### 利用商品的變數，擴增新的最小與最大的變數

In [63]:
for prod in products:
    print()
    print(prod)
    for begin, end in [(2,3),(2,5)]:
        prods = ["%s_prev%s" % (prod, i) for i in range(begin,end+1)]
        mp_df = train_df.as_matrix(columns=prods)
        print(prods)

        minf = "%s_min_%s_%s"%(prod,begin,end)
        train_df[minf] = np.nanmin(mp_df, axis=1).astype(np.int8)
        print (minf)
        maxf = "%s_max_%s_%s"%(prod,begin,end)
        train_df[maxf] = np.nanmax(mp_df, axis=1).astype(np.int8)
        print (maxf)
        features += (minf,maxf,)


ind_ahor_fin_ult1
['ind_ahor_fin_ult1_prev2', 'ind_ahor_fin_ult1_prev3']
ind_ahor_fin_ult1_min_2_3


/Users/liouscott/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: All-NaN axis encountered
  # Remove the CWD from sys.path while we load stuff.
/Users/liouscott/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: All-NaN slice encountered
  del sys.path[0]


ind_ahor_fin_ult1_max_2_3
['ind_ahor_fin_ult1_prev2', 'ind_ahor_fin_ult1_prev3', 'ind_ahor_fin_ult1_prev4', 'ind_ahor_fin_ult1_prev5']
ind_ahor_fin_ult1_min_2_5
ind_ahor_fin_ult1_max_2_5

ind_aval_fin_ult1
['ind_aval_fin_ult1_prev2', 'ind_aval_fin_ult1_prev3']
ind_aval_fin_ult1_min_2_3
ind_aval_fin_ult1_max_2_3
['ind_aval_fin_ult1_prev2', 'ind_aval_fin_ult1_prev3', 'ind_aval_fin_ult1_prev4', 'ind_aval_fin_ult1_prev5']
ind_aval_fin_ult1_min_2_5
ind_aval_fin_ult1_max_2_5

ind_cco_fin_ult1
['ind_cco_fin_ult1_prev2', 'ind_cco_fin_ult1_prev3']
ind_cco_fin_ult1_min_2_3
ind_cco_fin_ult1_max_2_3
['ind_cco_fin_ult1_prev2', 'ind_cco_fin_ult1_prev3', 'ind_cco_fin_ult1_prev4', 'ind_cco_fin_ult1_prev5']
ind_cco_fin_ult1_min_2_5
ind_cco_fin_ult1_max_2_5

ind_cder_fin_ult1
['ind_cder_fin_ult1_prev2', 'ind_cder_fin_ult1_prev3']
ind_cder_fin_ult1_min_2_3
ind_cder_fin_ult1_max_2_3
['ind_cder_fin_ult1_prev2', 'ind_cder_fin_ult1_prev3', 'ind_cder_fin_ult1_prev4', 'ind_cder_fin_ult1_prev5']
ind_cder_fin_ul

### 以下為過程觀察

In [64]:
features

('canal_entrada',
 'pais_residencia',
 'age',
 'renta_top',
 'renta',
 'antiguedad',
 'tipodom',
 'cod_prov',
 'fecha_dato_month',
 'fecha_dato_year',
 'fecha_alta_month',
 'fecha_alta_year',
 'dato_minus_alta',
 'indresi_n',
 'indext_s',
 'conyuemp_n',
 'sexo_h',
 'sexo_v',
 'ind_empleado_a',
 'ind_empleado_b',
 'ind_empleado_f',
 'ind_empleado_n',
 'ind_nuevo_new',
 'segmento_top',
 'segmento_particulares',
 'segmento_universitario',
 'indfall_s',
 'ind_actividad_cliente',
 'indrel_1',
 'indrel_99',
 'indrel_1mes',
 'tiprel_1mes_a',
 'tiprel_1mes_i',
 'tiprel_1mes_p',
 'tiprel_1mes_r',
 'ind_ahor_fin_ult1_prev1',
 'ind_aval_fin_ult1_prev1',
 'ind_cco_fin_ult1_prev1',
 'ind_cder_fin_ult1_prev1',
 'ind_cno_fin_ult1_prev1',
 'ind_ctju_fin_ult1_prev1',
 'ind_ctma_fin_ult1_prev1',
 'ind_ctop_fin_ult1_prev1',
 'ind_ctpp_fin_ult1_prev1',
 'ind_deco_fin_ult1_prev1',
 'ind_deme_fin_ult1_prev1',
 'ind_dela_fin_ult1_prev1',
 'ind_ecue_fin_ult1_prev1',
 'ind_fond_fin_ult1_prev1',
 'ind_hip_fin_u

### 統整所有變數並確認欄位是否重覆

In [65]:
leave_columns = ["ncodpers", "int_date", "fecha_dato"] + list(products) + list(features)
assert len(leave_columns) == len(set(leave_columns))
train_df = train_df[leave_columns]

In [66]:
len(leave_columns)

278

In [67]:
len(set(leave_columns))

278